# Game Design

In [276]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import clear_output

In [277]:
poeltl = pd.read_csv('poeltl.csv', skiprows=1).iloc[: , 1:]
poeltl = poeltl[['PLAYER', 'TEAM', 'AGE', 'HEIGHT']]
poeltl = poeltl.sort_values(by=['PLAYER'])

players = pd.read_csv('players.csv', skiprows=1).iloc[: , 1:]
players['PLAYER'] = players['PLAYER'].apply(lambda x : x[1:].replace('\n', ' '))
players = players[['PLAYER', 'NUMBER', 'POSITION']]
players = players[players['PLAYER'].isin(poeltl.PLAYER.values)]
players = players.sort_values(by=['PLAYER'])
players.index = np.arange(0, 596)

poeltl['NUMBER'] = players['NUMBER']
poeltl['POSITION'] = players['POSITION']

poeltl = poeltl.dropna()
poeltl['NUMBER'] = poeltl['NUMBER'].astype(int)

In [278]:
west = ['GSW', 'LAL', 'LAC', 'SAC', 'PHX', 'MEM', 'NOP', 'SAS', 'HOU', 'DAL', 'POR', 'DEN', 'UTA', 'MIN', 'OKC']
east = ['TOR', 'BOS', 'NYK', 'BKN', 'PHI', 'CLE', 'IND', 'DET', 'MIL', 'CHI', 'WAS', 'ORL', 'MIA', 'CHA', 'ATL']

atlantic = ['TOR', 'BOS', 'NYK', 'BKN', 'PHI']
central = ['CLE', 'IND', 'DET', 'MIL', 'CHI']
southeast = ['WAS', 'ORL', 'MIA', 'CHA', 'ATL']
northwest = ['POR', 'DEN', 'UTA', 'MIN', 'OKC']
pacific = ['GSW', 'LAL', 'LAC', 'SAC', 'PHX']
southwest = ['MEM', 'NOP', 'SAS', 'HOU', 'DAL']

divisions = ['Atl.', 'Cen.', 'SE', 'NW', 'Pac.', 'SW']
list_divisions = [atlantic, central, southeast, northwest, pacific, southwest]

def conference(team):
    if np.count_nonzero([(team == i) for i in west]) > 0:
        return "West"
    elif np.count_nonzero([(team == i) for i in east]) > 0:
        return "East"
    else:
        return "N/A"
    
def division(team):
    return divisions[ np.where([(team in div) for div in list_divisions])[0][0] ]

poeltl['Conference'] = poeltl['TEAM'].apply(conference)
poeltl['Division'] = poeltl['TEAM'].apply(division)

poeltl = poeltl[['PLAYER', 'TEAM', 'Conference', 'Division', 'POSITION', 'HEIGHT', 'AGE', 'NUMBER']]
poeltl_columns = ['', 'TEAM', 'CONF', 'DIV', 'POS', 'HT', 'AGE', '#']
poeltl.columns = poeltl_columns

poeltl_placeholder = poeltl

In [279]:
allstars = pd.read_csv("allstars.csv")
allstars['All-Star'] = True
allstars = allstars[['Player', 'All-Star']]
allstars['Player'] = allstars['Player'].apply(lambda s : s.split("*")[0])
allstars['Player'] = allstars['Player'].apply(lambda s : s.split("†")[0])
allstars['Player'] = allstars['Player'].apply(lambda s : s.split("§")[0])
allstars['Player'] = allstars['Player'].apply(lambda s : s.split("^")[0])
allstars['Player'] = allstars['Player'].apply(lambda s : s.split("[")[0])

all_star_placeholder = poeltl.merge(allstars, left_on='', right_on='Player').iloc[:, :-2]

In [280]:
def position_filter(df, position, status):
    if status == 'green':
        if position == 'G-F' or position == 'F-G':
            return df[(df['POS'] == 'G-F') | (df['POS'] == 'F-G')]
        elif position == 'C-F' or position == 'F-C':
            return df[(df['POS'] == 'C-F') | (df['POS'] == 'F-C')]
        else:
            return df[(df['POS'] == position)]
        
    elif status == 'yellow':
        if position == 'G':
            return df[(df['POS'] == 'G-F') | (df['POS'] == 'F-G')]
        elif position == 'F':
            return df[(df['POS'] == 'G-F') | (df['POS'] == 'F-G') | 
                          (df['POS'] == 'F-C') | (df['POS'] == 'C-F')]
        elif position == 'C':
            return df[(df['POS'] == 'C-F') | (df['POS'] == 'F-C')]
        elif position == 'F-C' or position == 'C-F':
            return df[(df['POS'] == 'F') | (df['POS'] == 'C')]
        elif position == 'G-F' or position == 'G-F':
            return df[(df['POS'] == 'G') | (df['POS'] == 'F')]
    
    else:
        if position == 'G-F' or position == 'F-G':
            return df[(df['POS'] != 'G-F') & (df['POS'] != 'F-G')]
        elif position == 'C-F' or position == 'F-C':
            return df[(df['POS'] != 'C-F') & (df['POS'] != 'F-C')]
        else:
            return df[(df['POS'] != position)]
    
def height_to_num(string):
    num = int(string[0]) + float(string[2:]) / 12
    return float( str(num)[0:5] )

def num_to_height(number):
    return str(number)[0] + "-" + str( round(12 * (number % 1)) )

def height_filter(df, height, status, greater):
    df['HT_filter'] = df['HT'].apply(height_to_num)
    
    if status == 'green':
        return df[(df['HT'] == height)][poeltl_columns]
        
    elif status == 'yellow' and greater:
        return df[(df['HT'] == num_to_height(height_to_num(height) + 1/12)) |
                      (df['HT'] == num_to_height(height_to_num(height) + 2/12))][poeltl_columns]

    elif status == 'yellow' and not greater:
        return df[(df['HT'] == num_to_height(height_to_num(height) - 1/12)) |
                      (df['HT'] == num_to_height(height_to_num(height) - 2/12))][poeltl_columns]
    
    else:
        if greater:
            return df[(df['HT_filter'] > float( str(height_to_num(height) + 1/6)[0:5] ))][poeltl_columns]
        else:
            return df[(df['HT_filter'] < float( str(height_to_num(height) - 1/6)[0:5] ))][poeltl_columns]

def age_filter(df, age, status, greater):
    if status == 'green':
        return df[(df.AGE == age)]
    
    elif status == 'yellow' and greater:
        return df[(df.AGE == (age + 1)) | (df.AGE == (age + 2))]
    elif status == 'yellow' and not greater:
        return df[(df.AGE == (age - 1)) | (df.AGE == (age - 2))]
    
    elif greater:
        return df[(df.AGE > (age + 2))]
    else:
        return df[(df.AGE < (age - 2))]

def number_filter(df, number, status, greater):
    if status == 'green':
        return df[(df['#'] == number)]
    
    elif status == 'yellow' and greater:
        return df[(df['#'] == (number + 1)) | (df['#'] == (number + 2))]
    elif status == 'yellow' and not greater:
        return df[(df['#'] == (number - 1)) | (df['#'] == (number - 2))]
    
    elif greater:
        return df[(df['#'] > (number + 2))]
    else:
        return df[(df['#'] < (number - 2))]

In [281]:
def poeltl_filter(player, team, conf, div, pos, ht, ht_comp, age, age_comp, num, num_comp):
    eligible = poeltl
    attributes = poeltl[(poeltl[''] == player)].iloc[0].values
    
    #1
    if team:
        eligible = eligible[(eligible.TEAM == attributes[1])]
    else:
        eligible = eligible[(eligible.TEAM != attributes[1])]
        
    #2
    if conf:
        eligible = eligible[(eligible.CONF == attributes[2])]
    else:
        eligible = eligible[(eligible.CONF != attributes[2])]
        
    #3
    if team:
        eligible = eligible[(eligible.DIV == attributes[3])]
    else:
        eligible = eligible[(eligible.DIV != attributes[3])]

    #4
    if pos == 'green':
        position_filter(eligible, attributes[4], 'green')
    elif pos == 'yellow':
        position_filter(eligible, attributes[4], 'yellow')
    else:
        position_filter(eligible, attributes[4], 'gray')
    
    #5
    if ht == 'green':
        eligible = height_filter(eligible, attributes[5], 'green', True)
    elif ht == 'yellow':
        if ht_comp:
            eligible = height_filter(eligible, attributes[5], 'yellow', True)
        else:
            eligible = height_filter(eligible, attributes[5], 'yellow', False)
    else:
        if ht_comp:
            eligible = height_filter(eligible, attributes[5], 'gray', True)
        else:
            eligible = height_filter(eligible, attributes[5], 'gray', False)
    
    #6
    if age == 'green':
        eligible = age_filter(eligible, attributes[6], 'green', True)
    elif age == 'yellow':
        if age_comp:
            eligible = age_filter(eligible, attributes[6], 'yellow', True)
        else:
            eligible = age_filter(eligible, attributes[6], 'yellow', False)
    else:
        if age_comp:
            eligible = age_filter(eligible, attributes[6], 'gray', True)
        else:
            eligible = age_filter(eligible, attributes[6], 'gray', False)
    
    #7
    if num == 'green':
        eligible = number_filter(eligible, attributes[7], 'green', True)
    elif num == 'yellow':
        if num_comp:
            eligible = number_filter(eligible, attributes[7], 'yellow', True)
        else:
            eligible = number_filter(eligible, attributes[7], 'yellow', False)
    else:
        if num_comp:
            eligible = number_filter(eligible, attributes[7], 'gray', True)
        else:
            eligible = number_filter(eligible, attributes[7], 'gray', False)
        
    
    return eligible.sort_values(by=['HT'])

In [282]:
correct = ['+', '+', '+', '+', '+', '+', '+', '+']

def check_guess(player, guess):
    player_info = list(poeltl[(poeltl[''] == player)].iloc[0])
    guess_info = list(poeltl[(poeltl[''] == guess)].iloc[0])

    if player == guess:
        return ['+', '+', '+', '+', '+', '+', '+', '+']
    else:
        results = ['', ]
        
        #1-3
        for x in range(1, 4):
            if player_info[x] == guess_info[x]:
                results.append('+')
            else:
                results.append('-')

        #4 position
        if player_info[4] == guess_info[4]:
            results.append('+')
        elif 'F' in guess_info[4] and 'G' in guess_info[4] and 'F' in player_info[4] and 'G' in player_info[4]:
            results.append('+')
        elif 'F' in guess_info[4] and 'C' in guess_info[4] and 'F' in player_info[4] and 'C' in player_info[4]:
            results.append('+')
        elif len(guess_info[4]) == 1 and guess_info[4] in player_info[4]:
            results.append('✓')
        elif player_info[4] in guess_info[4]:
            results.append('✓')
        else:
            results.append('-')
        
        #5 height
        if height_to_num(player_info[5]) == height_to_num(guess_info[5]):
            results.append('+')
        elif height_to_num(player_info[5]) > height_to_num(guess_info[5]):
            if height_to_num(player_info[5]) > float( str(height_to_num(guess_info[5]) + 1/6)[0:5] ):
                results.append('- ∧')
            else:
                results.append('✓ ∧')
        else:
            if height_to_num(player_info[5]) < float( str(height_to_num(guess_info[5]) - 1/6)[0:5] ):
                results.append('- V')
            else:
                results.append('✓ V')
        
        #6-7
        for x in range(6, 8):
            if player_info[x] == guess_info[x]:
                results.append('+')
            elif player_info[x] > guess_info[x]:
                if player_info[x] > guess_info[x] + 2:
                    results.append('- ∧')
                else:
                    results.append('✓ ∧')
            else:
                if player_info[x] < guess_info[x] - 2:
                    results.append('- V')
                else:
                    results.append('✓ V')
        
        return results

In [283]:
def make_indices(n):
    indices = []
    for x in range(1, n + 1):
        indices.append(int(np.floor((x + 1) / 2)))
    return indices

In [284]:
from pandas.core.arrays.numeric import T
button_add = widgets.Button(description="Guess Player")
all_star_button_add = widgets.Button(description="Guess Player")
button_newGame = widgets.Button(description="New Game")
all_star_button_newGame = widgets.Button(description="New Game")
output = widgets.Output()

mystery_player = poeltl.sample()[''].values[0]
roster_df = poeltl[(poeltl.TEAM == 0)]

def game_setup():
    clear_output()
    display(button_newGame, output)
    print("Try to guess the mystery NBA player!\n+ indicates a match\n✓ in the POS column indicates a partial match\n✓ in any other column indicates this attribute is within 2 inches/years/numbers of the mystery player)\n∧ or V indicates if this attribute is greater than or less than the mystery player")
    interact(choose_team, Team=teams)
    display(button_add, output)
        
    return None

def all_star_game_setup():
    clear_output()
    display(all_star_button_newGame, output)
    print("Try to guess the mystery NBA player!\n+ indicates a match\n✓ in the POS column indicates a partial match\n✓ in any other column indicates this attribute is within 2 inches/years/numbers of the mystery player)\n∧ or V indicates if this attribute is greater than or less than the mystery player")
    interact(choose_team, Team=teams)
    display(all_star_button_add, output)
        
    return None

def add_player(a):
    global roster_df
    
    results = check_guess(mystery_player, selected_player)
    
    if results == correct:
        roster_df.loc[len(roster_df)] = list(poeltl[(poeltl[''] == selected_player)].iloc[0])
        roster_df.loc[len(roster_df)] = check_guess(mystery_player, selected_player)
        roster_df.index = make_indices(len(roster_df))
        
        clear_output()
        display(button_newGame, output)
        print("Try to guess the mystery NBA player!\n+ indicates a match\n✓ in the POS column indicates a partial match\n✓ in any other column indicates this attribute is within 2 inches/years/numbers of the mystery player)\n∧ or V indicates if this attribute is greater than or less than the mystery player")
        display(roster_df)
        if np.max(roster_df.index) == 1:
            print("Congratulations! You correctly guessed the mystery player in 1 guess!")
        else:
            print("Congratulations! You correctly guessed the mystery player in", np.max(roster_df.index), "guesses!")
        
    else:
        roster_df.loc[len(roster_df)] = list(poeltl[(poeltl[''] == selected_player)].iloc[0])
        roster_df.loc[len(roster_df)] = check_guess(mystery_player, selected_player)
        roster_df.index = make_indices(len(roster_df))
    
        game_setup()
        display(roster_df)
    
    return None

def all_star_add_player(a):
    global roster_df
    
    results = check_guess(mystery_player, selected_player)
    
    if results == correct:
        roster_df.loc[len(roster_df)] = list(poeltl[(poeltl[''] == selected_player)].iloc[0])
        roster_df.loc[len(roster_df)] = check_guess(mystery_player, selected_player)
        roster_df.index = make_indices(len(roster_df))
        
        clear_output()
        display(button_newGame, output)
        print("Try to guess the mystery NBA player!\n+ indicates a match\n✓ in the POS column indicates a partial match\n✓ in any other column indicates this attribute is within 2 inches/years/numbers of the mystery player)\n∧ or V indicates if this attribute is greater than or less than the mystery player")
        display(roster_df)
        if np.max(roster_df.index) == 1:
            print("Congratulations! You correctly guessed the mystery player in 1 guess!")
        else:
            print("Congratulations! You correctly guessed the mystery player in", np.max(roster_df.index), "guesses!")
        
    else:
        roster_df.loc[len(roster_df)] = list(poeltl[(poeltl[''] == selected_player)].iloc[0])
        roster_df.loc[len(roster_df)] = check_guess(mystery_player, selected_player)
        roster_df.index = make_indices(len(roster_df))
    
        all_star_game_setup()
        display(roster_df)
    
    return None

teams = np.concatenate((np.array(['All Teams']), np.sort(pd.unique(poeltl.TEAM))))
teams_placeholder = teams
players = np.sort(poeltl[''].values)

def new_game(a):
    global mystery_player
    global roster_df
    global teams
    global player
    global poeltl
    
    poeltl = poeltl_placeholder
    teams = teams_placeholder
    players = np.sort(poeltl[''].values)   
    mystery_player = poeltl.sample()[''].values[0]
    roster_df = poeltl[(poeltl.TEAM == 0)]
    
    game_setup()
        
    return None

def all_star_new_game(a):
    global mystery_player
    global roster_df
    global teams
    global poeltl
    global players

    poeltl = all_star_placeholder

    teams = teams_placeholder
    players = np.sort(poeltl[''].values)   
    mystery_player = poeltl.sample()[''].values[0]
    roster_df = poeltl[(poeltl.TEAM == 0)]
    
    all_star_game_setup()
        
    return None

button_add.on_click(add_player)
all_star_button_add.on_click(all_star_add_player)
button_newGame.on_click(new_game)
all_star_button_newGame.on_click(all_star_new_game)
    
def choose_team(Team):
    global selected_team
    global players

    selected_team = Team
    
    if selected_team == 'All Teams':
        players = np.sort(poeltl[''].values)
    else:
        players = list(poeltl[poeltl['TEAM'] == selected_team][''])

    interact(choose_player, Player=players)

    return None

def choose_player(Player):
    global selected_player

    selected_player = Player

    return None

def poeltl_game():
    display(button_newGame, output)
    print("Try to guess the mystery NBA player!\n+ indicates a match\n✓ in the POS column indicates a partial match\n✓ in any other column indicates this attribute is within 2 inches/years/numbers of the mystery player)\n∧ or V indicates if this attribute is greater than or less than the mystery player")
    interact(choose_team, Team=teams)
    display(button_add, output)

def all_star_poeltl_game():
    display(all_star_button_newGame, output)
    print("Try to guess the mystery NBA player!\n+ indicates a match\n✓ in the POS column indicates a partial match\n✓ in any other column indicates this attribute is within 2 inches/years/numbers of the mystery player)\n∧ or V indicates if this attribute is greater than or less than the mystery player")
    interact(choose_team, Team=teams)
    display(button_add, output)

In [285]:
eligible = poeltl
teams = np.concatenate((np.array(['All Teams']), np.sort(pd.unique(poeltl.TEAM))))
players = np.sort(poeltl[''].values)

def cheat_code(mystery_player, selected_player):
    results = check_guess(mystery_player, selected_player)
    attributes = poeltl[(poeltl[''] == selected_player)].values[0]
    
    #1
    if results[1] == '-':
        today = eligible[(eligible.TEAM != attributes[1])]
    else:
        today = eligible[(eligible.TEAM == attributes[1])]
        
    #2
    if results[2] == '-':
        today = today[(today.CONF != attributes[2])]
    else:
        today = today[(today.CONF == attributes[2])]
    
    #3
    if results[3] == '-':
        today = today[(today.DIV != attributes[3])]
    else:
        today = today[(today.DIV == attributes[3])]
    
    #4
    if results[4] == '+':
        today = position_filter(today, attributes[4], 'green')
    elif results[4] == '✓':
        today = position_filter(today, attributes[4], 'yellow')
    elif results[4] == '-':
        today = position_filter(today, attributes[4], 'gray')
    else:
        pass
    
    #5-7
    filter_functions = [height_filter, age_filter, number_filter]
    for x in range(5, 8):
        if results[x] == '+':
            today = filter_functions[x - 5](today, attributes[x], 'green', True)
        elif results[x] == '✓ ∧':
            today = filter_functions[x - 5](today, attributes[x], 'yellow', True)
        elif results[x] == '✓ V':
            today = filter_functions[x - 5](today, attributes[x], 'yellow', False)
        elif results[x] == '- ∧':
            today = filter_functions[x - 5](today, attributes[x], 'gray', True)
        elif results[x] == '- V':
            today = filter_functions[x - 5](today, attributes[x], 'gray', False)
        else:
            pass
    
    return today.sort_values(by=['HT'])

In [286]:
easy_button_add = widgets.Button(description="Guess Player")
easy_button_newGame = widgets.Button(description="New Game")

def easy_choose_team(Team):
    global selected_team
    global players

    selected_team = Team
    
    if selected_team == 'All Teams':
        players = np.sort(eligible[''].values)
    else:
        players = list(eligible[eligible['TEAM'] == selected_team][''])

    interact(choose_player, Player=players)

    return None

def easy_game_setup():
    clear_output()
    display(easy_button_newGame, output)
    print("Try to guess the mystery NBA player!\n+ indicates a match\n✓ in the POS column indicates a partial match\n✓ in any other column indicates this attribute is within 2 inches/years/numbers of the mystery player)\n∧ or V indicates if this attribute is greater than or less than the mystery player")
    interact(easy_choose_team, Team=teams)
    display(easy_button_add, output)

    return None

def easy_add_player(a):
    global roster_df
    global eligible
    global teams
    global players
    
    results = check_guess(mystery_player, selected_player)
    
    if results == correct:
        roster_df.loc[len(roster_df)] = list(poeltl[(poeltl[''] == selected_player)].iloc[0])
        roster_df.loc[len(roster_df)] = check_guess(mystery_player, selected_player)
        roster_df.index = make_indices(len(roster_df))
        
        clear_output()
        display(easy_button_newGame, output)
        print("Try to guess the mystery NBA player!\n+ indicates a match\n✓ in the POS column indicates a partial match\n✓ in any other column indicates this attribute is within 2 inches/years/numbers of the mystery player)\n∧ or V indicates if this attribute is greater than or less than the mystery player")
        display(roster_df)
        if np.max(roster_df.index) == 1:
            print("Congratulations! You correctly guessed the mystery player in 1 guess!")
        else:
            print("Congratulations! You correctly guessed the mystery player in", np.max(roster_df.index), "guesses!")
        
    else:
        roster_df.loc[len(roster_df)] = list(poeltl[(poeltl[''] == selected_player)].iloc[0])
        roster_df.loc[len(roster_df)] = check_guess(mystery_player, selected_player)
        roster_df.index = make_indices(len(roster_df))
        
        eligible = cheat_code(mystery_player, selected_player)
        teams = np.concatenate((np.array(['All Teams']), np.sort(pd.unique(eligible.TEAM))))
        players = np.sort(eligible[''].values)
    
        easy_game_setup()
        display(roster_df)
    
    return None

def easy_new_game(a):
    global mystery_player
    global roster_df
    global eligible
    global teams
    global players
    global poeltl

    poeltl = poeltl_placeholder
    
    mystery_player = poeltl.sample()[''].values[0]
    roster_df = poeltl[(poeltl.TEAM == 0)]
    
    eligible = poeltl
    teams = np.concatenate((np.array(['All Teams']), np.sort(pd.unique(poeltl.TEAM))))
    players = np.sort(poeltl[''].values)
    
    easy_game_setup()
        
    return None

easy_button_add.on_click(easy_add_player)
easy_button_newGame.on_click(easy_new_game)

def easy_poeltl_game():
    display(easy_button_newGame, output)
    print("Try to guess the mystery NBA player!\n+ indicates a match\n✓ in the POS column indicates a partial match\n✓ in any other column indicates this attribute is within 2 inches/years/numbers of the mystery player)\n∧ or V indicates if this attribute is greater than or less than the mystery player")
    interact(easy_choose_team, Team=teams)
    display(easy_button_add, output)

# Poeltl Game

To play the game, select "Runtime" in the navigation bar above and click "Run all.

<br>

You can also manually load the game by: 

1) clicking the triangle to the left of where it says "11 cells hidden" above

2) clicking the triangle to the left of where it says "poeltl = ..." below.

In [287]:
poeltl = poeltl_placeholder
mystery_player = poeltl.sample()[''].values[0]

poeltl_game()

Button(description='New Game', style=ButtonStyle())

Output()

Try to guess the mystery NBA player!
+ indicates a match
✓ in the POS column indicates a partial match
✓ in any other column indicates this attribute is within 2 inches/years/numbers of the mystery player)
∧ or V indicates if this attribute is greater than or less than the mystery player


interactive(children=(Dropdown(description='Team', options=('All Teams', 'ATL', 'BKN', 'BOS', 'CHA', 'CHI', 'C…

Button(description='Guess Player', style=ButtonStyle())

Output()

# All-Star Mode (non all-stars are excluded)

To play the game, select "Runtime" in the navigation bar above and click "Run all.

<br>

You can also manually load the game by: 

1) clicking the triangle to the left of where it says "11 cells hidden" above

2) clicking the triangle to the left of where it says "poeltl = ..." below.

In [288]:
poeltl = poeltl.merge(allstars, left_on='', right_on='Player').iloc[:, :-2]
mystery_player = poeltl.sample()[''].values[0]

all_star_poeltl_game()

Button(description='New Game', style=ButtonStyle())

Output()

Try to guess the mystery NBA player!
+ indicates a match
✓ in the POS column indicates a partial match
✓ in any other column indicates this attribute is within 2 inches/years/numbers of the mystery player)
∧ or V indicates if this attribute is greater than or less than the mystery player


interactive(children=(Dropdown(description='Team', options=('All Teams', 'ATL', 'BKN', 'BOS', 'CHA', 'CHI', 'C…

Button(description='Guess Player', style=ButtonStyle())

Output()

# Easy Mode (only eligible players can be guessed)

To play the game, select "Runtime" in the navigation bar above and click "Run all.

<br>

You can also manually load the game by: 

1) clicking the triangle to the left of where it says "11 cells hidden" above

2) clicking the triangle to the left of where it says "easy_poeltl_game()" below.

In [289]:
poeltl = poeltl_placeholder
mystery_player = poeltl.sample()[''].values[0]

easy_poeltl_game()

Button(description='New Game', style=ButtonStyle())

Output()

Try to guess the mystery NBA player!
+ indicates a match
✓ in the POS column indicates a partial match
✓ in any other column indicates this attribute is within 2 inches/years/numbers of the mystery player)
∧ or V indicates if this attribute is greater than or less than the mystery player


interactive(children=(Dropdown(description='Team', options=('All Teams', 'ATL', 'BKN', 'BOS', 'CHA', 'CHI', 'C…

Button(description='Guess Player', style=ButtonStyle())

Output()